# Computation for Theorem 3.11

In [1]:
import os
import pandas as pd
from inspect import getsource
from utils import *
if not os.path.exists('data/'):
    os.makedirs('data')

Based on Lemma 3.9, the following code computes the interval which the value of $\log(N)$ cannot belong to, and then provides an upper bound of $\log(N)$.

Here $N = |abc|/\gcd(16,abc)$, so $\log(|abc|) \le \log(N)+\log(16)$, hence we can also get an upper bound of $\log(abc)$.
We can take this upper bound [enlarging slightly for the error in computation] as an upper bound of $f(u,t)$.

Note that if we get the upper bound $f(u,t) \le M$ by the compute using the first definition of $b_1$ in Lemma 3.9, then for any $u' \ge u$, $s' \ge s$, we must have $f(u', t') \le M$ [since $b_1$ is monotonically increasing when $r, s, t$ increase].

In [2]:
if not os.path.isfile('data/list_of_f2_1e4.pkl'):
    list_of_f2_1e4 = list_of_f2(pow(10,4))
    pd.to_pickle(list_of_f2_1e4,"data/list_of_f2_1e4.pkl")
else:
    list_of_f2_1e4 = pd.read_pickle("data/list_of_f2_1e4.pkl")

if not os.path.isfile('data/volume_150_when_bad_at_2.pkl'):
    volume_150_when_bad_at_2 = compute_log_volume(p, is_bad_at_2=True)
    pd.to_pickle(volume_150_when_bad_at_2,"data/volume_150_when_bad_at_2.pkl")
else:
    volume_150_when_bad_at_2 = pd.read_pickle("data/volume_150_when_bad_at_2.pkl")

100%|██████████| 30001/30001 [00:00<00:00, 1760620.59it/s]


In [3]:
def bound_of_log_N(u, t, is_second_b1 = False, s = 0):
    is_bad_at_2 = min(u,t) > 4
    my_log_volumes = list_of_f2_1e4.copy()
    if is_bad_at_2:
        for p in volume_150_when_bad_at_2:
            my_log_volumes[p] = volume_150_when_bad_at_2[p]

    if min(u,t) >= 4:
        my_primes = [p for p in primes(2000) if p >= 11]
    else:
        my_primes = [p for p in primes(2000) if p >= 23 and p not in (37,43,67,163)]
    intervals = []
    arr_S = []
    for n in range(2,20):
        for k in range(len(my_primes)-n):
            arr_S.append(my_primes[k:k+n])
    for k in range(len(arr_S)):
        S = sorted(list(arr_S[k]))
        n = len(S)
        a1 = 0 # avg of O(1/p)
        a2 = 0 # avg of log-volume
        a3 = 0 # sum of log(p)
        for p in S:
            a1 += (11*p+31) / (p*p+p-12)
            a2 += my_log_volumes[p]  
            a3 += log(p)
        a1 = a1 / n
        a2 = 3 * a2 / n
        a3 = a3

        p0 = S[0]
        b1 = max(2/n+(3+a1)/p0, ((3+a1)/min(u,t)+2/n+(3+a1)/p0)/2, (3+a1)*(1/u+1/t)/2 )
        if u==t and is_second_b1:
            b1 = max(2/n+(3+a1)/p0, (2/n+(3+a1)/p0)*(t-1)/(3*t-1) + (3+a1)*2/(3*t-1), (3+a1)*(2+(t-1)/s)/(3*t-1))
        b2 = a2 + (3+a1)*(a3+log(16)/min(u,t))

        if b1>=1:
            continue
        upper_bound = p0*S[1]*log(2)
        lower_bound = b2 / (1-b1)
        if lower_bound > upper_bound:
            continue
        intervals.append([lower_bound,upper_bound])

    res = merge_intervals(intervals)
    return res

def bound_of_log_abc(u, t, is_second_b1 = False, s = 0):
    intervals = bound_of_log_N(u, t, is_second_b1, s)
    if len(intervals) == 0:
        return -1000
    if intervals[-1][1] <= 10^6:
        # check that the interval of impossible values of \log(N) covers 10^6 
        print("Error for existing possible \log(N) near 10^6")
    return intervals[-1][0] + log(16)    

In [4]:
# assume without loss of generality that u <= t
for u in range(2,11):
    tmp_bound = 0
    for t in range(u,15+(u==2)*15):
        tmp = ceil(bound_of_log_abc(u,t)+0.1)
        if tmp < 0:
            continue
        if tmp_bound == tmp:
            break
        else:
            tmp_bound = tmp
        print(f'$({u},{t})$ & ${ceil(bound_of_log_abc(u,t)+0.1)}$ \\\\')

$(2,7)$ & $62642$ \\
$(2,8)$ & $22599$ \\
$(2,9)$ & $13997$ \\
$(2,10)$ & $10441$ \\
$(2,11)$ & $8767$ \\
$(2,12)$ & $7650$ \\
$(2,13)$ & $6885$ \\
$(2,14)$ & $6453$ \\
$(2,15)$ & $6253$ \\
$(2,16)$ & $6069$ \\
$(2,17)$ & $5892$ \\
$(2,18)$ & $5835$ \\
$(2,19)$ & $5822$ \\
$(3,4)$ & $5753$ \\
$(3,5)$ & $2710$ \\
$(3,6)$ & $1982$ \\
$(3,7)$ & $1643$ \\
$(3,8)$ & $1521$ \\
$(3,9)$ & $1477$ \\
$(3,10)$ & $1465$ \\
$(4,4)$ & $1760$ \\
$(4,5)$ & $1174$ \\
$(4,6)$ & $999$ \\
$(4,7)$ & $919$ \\
$(4,8)$ & $896$ \\
$(4,9)$ & $895$ \\
$(5,5)$ & $810$ \\
$(5,6)$ & $711$ \\
$(5,7)$ & $675$ \\
$(5,8)$ & $672$ \\
$(6,6)$ & $595$ \\
$(6,7)$ & $569$ \\
$(7,7)$ & $541$ \\
$(7,8)$ & $528$ \\
$(8,8)$ & $516$ \\
$(9,9)$ & $511$ \\
$(10,10)$ & $511$ \\


In [5]:
for t in range(3,11):
    tmp_bound = 0
    for s in range(t,15+(t==3)*5):
        tmp = ceil(bound_of_log_abc(t,t,True,s)+0.1)
        if tmp < 0:
            continue
        if tmp_bound == tmp:
            break
        else:
            tmp_bound = tmp
        print(f'$({t},{s},{t})$ & ${ceil(bound_of_log_abc(t,t,True,s)+0.1)}$ \\\\')

$(3,4,3)$ & $20581$ \\
$(3,5,3)$ & $9174$ \\
$(3,6,3)$ & $6269$ \\
$(3,7,3)$ & $5132$ \\
$(3,8,3)$ & $4613$ \\
$(3,9,3)$ & $4293$ \\
$(3,10,3)$ & $4127$ \\
$(3,11,3)$ & $4015$ \\
$(3,12,3)$ & $3897$ \\
$(3,13,3)$ & $3892$ \\
$(4,4,4)$ & $1760$ \\
$(4,5,4)$ & $1468$ \\
$(4,6,4)$ & $1308$ \\
$(4,7,4)$ & $1252$ \\
$(4,8,4)$ & $1249$ \\
$(5,5,5)$ & $810$ \\
$(5,6,5)$ & $738$ \\
$(5,7,5)$ & $732$ \\
$(6,6,6)$ & $595$ \\
$(6,7,6)$ & $579$ \\
$(7,7,7)$ & $541$ \\
$(7,8,7)$ & $533$ \\
$(8,8,8)$ & $516$ \\
$(9,9,9)$ & $511$ \\
$(10,10,10)$ & $511$ \\


Based on Lemma 3.10, the following code computes the interval which the value of $\log(N)$ cannot belong to, and then provides an upper bound of $\log(N)$.

Here $c = z^m, N = |c|/\gcd(1728,c)$, so $\log(|z^m|) = \log(c) \le \log(N)+\log(1728)$, hence we can also get an upper bound [enlarging slightly for the error in computation] of $\log(c)$.

In [6]:
if not os.path.isfile('data/list_of_f3_1e4.pkl'):
    list_of_f3_1e4 = list_of_f3(pow(10,4))
    pd.to_pickle(list_of_f3_1e4,"data/list_of_f3_1e4.pkl")
else:
    list_of_f3_1e4 = pd.read_pickle("data/list_of_f3_1e4.pkl")

100%|██████████| 300002/300002 [00:00<00:00, 1713445.56it/s]


In [7]:
def bound2_of_log_N(m): 
    my_log_volumes = list_of_f3_1e4.copy()
    my_primes = [p for p in primes(3000) if p >= 23 and p not in (37,43,67,163)]
    intervals = []
    arr_S = []
    for n in range(2,20):
        for k in range(len(my_primes)-n):
            arr_S.append(my_primes[k:k+n])
    for k in range(len(arr_S)):
        S = sorted(list(arr_S[k]))
        n = len(S)
        a1 = 0 # avg of O(1/p)
        a2 = 0 # avg of log-volume
        a3 = 0 # sum of log(p)
        for p in S:
            a1 += (119*p+355) / (5*p*p+5*p-60)
            a2 += my_log_volumes[p]  
            a3 += log(p)
        a1 = a1 / n
        a2 = 6 * a2 / n
        a3 = a3

        p0 = S[0]
        b1 = max(2/n+(6+a1)/p0, (6+a1)/m)
        b2 = a2 + (6+a1)*(a3 + log(1728)/m)

        if b1>=1:
            continue
        upper_bound = p0*S[1]*log(2)
        lower_bound = b2 / (1-b1)
        if lower_bound > upper_bound:
            continue
        intervals.append([lower_bound,upper_bound])

    res = merge_intervals(intervals)
    return res

def bound2_of_log_abc(m):
    intervals = bound2_of_log_N(m)
    if len(intervals) == 0:
        return -1
    return intervals[-1][0] + log(1728)    

In [8]:
for m in range(7,50):
    print(f'${m}$ & ${ceil(bound2_of_log_abc(m)+0.1)}$', '&' if m%5!=1 else '\\\\')

$7$ & $611393$ &
$8$ & $202114$ &
$9$ & $116111$ &
$10$ & $81135$ &
$11$ & $63249$ \\
$12$ & $55529$ &
$13$ & $45838$ &
$14$ & $42312$ &
$15$ & $38985$ &
$16$ & $34337$ \\
$17$ & $33222$ &
$18$ & $31783$ &
$19$ & $30947$ &
$20$ & $30630$ &
$21$ & $30002$ \\
$22$ & $29514$ &
$23$ & $29355$ &
$24$ & $28151$ &
$25$ & $28095$ &
$26$ & $27811$ \\
$27$ & $27662$ &
$28$ & $27588$ &
$29$ & $27396$ &
$30$ & $27396$ &
$31$ & $27246$ \\
$32$ & $27246$ &
$33$ & $26536$ &
$34$ & $26532$ &
$35$ & $26524$ &
$36$ & $26507$ \\
$37$ & $26507$ &
$38$ & $26507$ &
$39$ & $26507$ &
$40$ & $26507$ &
$41$ & $26507$ \\
$42$ & $26507$ &
$43$ & $26507$ &
$44$ & $26507$ &
$45$ & $26507$ &
$46$ & $26507$ \\
$47$ & $26507$ &
$48$ & $26507$ &
$49$ & $26507$ &
